# Análisis de vuelos
_____
# Limpieza y transformación

En este proyecto se va a proceder con la limpieza del dataset obtenido en la primer etapa.
También se procede a la ingeniería de features para poder realizar reportes con valores como hora, nes.

In [1]:
import pandas as pd


def imprimir_reporte(titulo, datos):
    ancho = max(len(titulo), 25) # Ajusta el ancho mínimo a 25 o al largo del título
    print("=" * 40)
    print(titulo.upper())
    print("-" * ancho)
    print(datos)
    print("=" * 40)


def basic_report_df(df, title):
    print("=" * 40)
    print("Report:\t", title)

    imprimir_reporte("Shape:", f"Columns: {df.shape[1]}\t|\tRows: {df.shape[0]}")
    imprimir_reporte("Info", df.info())

    print_df(df.describe().T)

    imprimir_reporte("Null values count:", df.isnull().sum())

    imprimir_reporte("Unique values:", df.nunique())


def print_df(df_desc):
    from tabulate import tabulate

    # 'headers' son las columnas (count, mean, std, etc.)
    # 'tablefmt' puede ser "grid", "fancy_grid", "pipe" o "psql"
    print(tabulate(df_desc, headers='keys', tablefmt='psql'))

def read_csv_file(filename):
    return pd.read_csv(filename)


In [2]:
df_jetsmart = read_csv_file("vuelos_anual_WJ_consolidado_2025.csv")
basic_report_df(df_jetsmart, "Jetsmart")

Report:	 Jetsmart
SHAPE:
-------------------------
Columns: 8	|	Rows: 23645
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23645 entries, 0 to 23644
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Vuelo               23645 non-null  object
 1   Ruta                23645 non-null  object
 2   Hora Programada     23645 non-null  object
 3   Hora Real           23306 non-null  object
 4   Demora en despegar  23645 non-null  object
 5   fecha               23645 non-null  object
 6   mes                 23645 non-null  int64 
 7   empresa             23645 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.4+ MB
INFO
-------------------------
None
+-----+---------+--------+---------+-------+-------+-------+-------+-------+
|     |   count |   mean |     std |   min |   25% |   50% |   75% |   max |
|-----+---------+--------+---------+-------+-------+-------+-------+-------|
| mes |   23645 | 6

In [3]:
df_flybondi = read_csv_file("vuelos_anual_FO_consolidado 2025.csv")
basic_report_df(df_flybondi, "Flybondi")

Report:	 Flybondi
SHAPE:
-------------------------
Columns: 8	|	Rows: 20186
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20186 entries, 0 to 20185
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Vuelo               20186 non-null  object
 1   Ruta                20186 non-null  object
 2   Hora Programada     20186 non-null  object
 3   Hora Real           18909 non-null  object
 4   Demora en despegar  20186 non-null  object
 5   fecha               20186 non-null  object
 6   mes                 20186 non-null  int64 
 7   empresa             20186 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.2+ MB
INFO
-------------------------
None
+-----+---------+---------+---------+-------+-------+-------+-------+-------+
|     |   count |    mean |     std |   min |   25% |   50% |   75% |   max |
|-----+---------+---------+---------+-------+-------+-------+-------+-------|
| mes |   20186 

## Valores nulos
En la columna de "Hora Real" se encuentran más de mil valores nulos, esto es debido a que si existen vuelos cancelados, no existe el valor.

## Limpieza de datos

Transformación de datos


In [4]:
import pandas as pd
import re

def transformar_demoras(df, columna_nombre):
    def procesar_fila(valor):
        texto = str(valor).lower().strip()

        # 1. Salidas rápidas (Early exits)
        if 'cancelado' in texto:
            return None, 'Cancelled'
        if 'a tiempo' in texto:
            return 0, 'On time'

        # 2. Extracción de tiempo usando Regex
        # Buscamos patrones de horas (h/hs) y minutos (min)
        h = re.search(r'(\d+)\s*hs?', texto)
        m = re.search(r'(\d+)\s*min', texto)

        total_minutos = int(h.group(1)) * 60 if h else 0
        total_minutos += int(m.group(1)) if m else 0

        # 3. Clasificación lógica
        if 'adelantado' in texto:
            return -total_minutos, 'Early'

        if 'tarde' in texto or total_minutos > 0:
            return total_minutos, 'Delayed'

        return 0, 'On time'

    # Aplicación eficiente
    # Usar result_type='expand' con apply es más directo que convertir a lista y luego a DataFrame
    df[['minutos_netos_demora', 'status']] = df[columna_nombre].apply(
        lambda x: pd.Series(procesar_fila(x))
    )

    return df

def procesar_vuelos(df, col_demora, col_hora_prog):
    """
    Realiza toda la limpieza y categorización de demoras en un solo paso.
    """
    # 1. Aplicar la transformación de minutos y status (usando la función anterior)
    df = transformar_demoras(df, col_demora)

    # 2. Flag de cancelación
    df['is_cancelled'] = df[col_demora].str.contains('Cancelado', na=False, case=False)

    # 3. Procesamiento de tiempo
    df[col_hora_prog] = pd.to_datetime(df[col_hora_prog], errors='coerce')
    df['franja_horaria'] = df[col_hora_prog].dt.hour

    # 4. Categorización por niveles (Bins)
    bins = [-float('inf'), 0, 15, 45, 180, float('inf')]
    labels = [
        'A tiempo/Adelantado',
        'Demora Leve (0-15m)',
        'Demora Media (15-45m)',
        'Demora Grave (45m-3h)',
        'Crítico (>3h)'
    ]

    df['nivel_demora'] = pd.cut(df['minutos_netos_demora'], bins=bins, labels=labels)
    df['nivel_demora'] = df['nivel_demora'].astype(str).replace('nan', 'Cancelado')
    orden_logico = ['A tiempo/Adelantado', 'Demora Leve (0-15m)', 'Demora Media (15-45m)',
                'Demora Grave (45m-3h)', 'Crítico (>3h)', 'Cancelado']

    df['nivel_demora'] = pd.Categorical(df['nivel_demora'], categories=orden_logico, ordered=True)

    return df

def transform_column_to_datetime(df, col_name):
    df[col_name] = pd.to_datetime(df[col_name], errors='coerce')
    return df

In [5]:
df_jetsmart_final = procesar_vuelos(df_jetsmart, 'Demora en despegar', 'Hora Programada')
df_jetsmart_final

/tmp/ipython-input-4279419606.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col_hora_prog] = pd.to_datetime(df[col_hora_prog], errors='coerce')


,Vuelo,Ruta,Hora Programada,Hora Real,Demora en despegar,fecha,mes,empresa,minutos_netos_demora,status,is_cancelled,franja_horaria,nivel_demora
0,WJ 3169,Aeroparque → Neuquen,2026-02-19 18:29:00,00:55 +1,6hs 26min tarde,2025-01-01,1,WJ,386.0,Delayed,False,18,Crítico (>3h)
1,WJ 3142,Aeroparque → Iguazú,2026-02-19 19:20:00,22:25,3hs 5min tarde,2025-01-01,1,WJ,185.0,Delayed,False,19,Crítico (>3h)
2,WJ 3145,Iguazú → Ezeiza,2026-02-19 21:41:00,00:38 +1,2hs 57min tarde,2025-01-01,1,WJ,177.0,Delayed,False,21,Demora Grave (45m-3h)
3,WJ 3165,Aeroparque → Neuquen,2026-02-19 16:40:00,18:46,2hs 6min tarde,2025-01-01,1,WJ,126.0,Delayed,False,16,Demora Grave (45m-3h)
4,WJ 3820,Aeroparque → Florianopolis,2026-02-19 16:55:00,18:56,2hs 1min tarde,2025-01-01,1,WJ,121.0,Delayed,False,16,Demora Grave (45m-3h)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23640,WJ 3011,Salta → Aeroparque,2026-02-19 21:29:00,21:24,adelantado 5min,2025-12-31,12,WJ,-5.0,Early,False,21,A tiempo/Adelantado
23641,WJ 3145,Iguazú → Ezeiza,2026-02-19 13:57:00,13:50,adelantado 7min,2025-12-31,12,WJ,-7.0,Early,False,13,A tiempo/Adelantado
23642,WJ 3056,Bariloche → Ezeiza,2026-02-19 22:52:00,22:42,adelantado 10min,2025-12-31,12,WJ,-10.0,Early,False,22,A tiempo/Adelantado
23643,WJ 3155,Iguazú → Ezeiza,2026-02-19 22:34:00,22:16,adelantado 18min,2025-12-31,12,WJ,-18.0,Early,False,22,A tiempo/Adelantado


In [6]:
df_flybondi_final = procesar_vuelos(df_flybondi, 'Demora en despegar', 'Hora Programada')
df_flybondi_final

/tmp/ipython-input-4279419606.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col_hora_prog] = pd.to_datetime(df[col_hora_prog], errors='coerce')


,Vuelo,Ruta,Hora Programada,Hora Real,Demora en despegar,fecha,mes,empresa,minutos_netos_demora,status,is_cancelled,franja_horaria,nivel_demora
0,FO 5912,Aeroparque → Rio de Janeiro,2026-02-19 13:05:00,NaN,Cancelado,2025-01-01,1,FO,NaN,Cancelled,True,13,Cancelado
1,FO 5237,Bariloche → Aeroparque,2026-02-19 07:50:00,19:22,11hs 32min tarde,2025-01-01,1,FO,692.0,Delayed,False,7,Crítico (>3h)
2,FO 5236,Ezeiza → Bariloche,2026-02-19 05:00:00,15:16,10hs 16min tarde,2025-01-01,1,FO,616.0,Delayed,False,5,Crítico (>3h)
3,FO 5027,Córdoba → Ezeiza,2026-02-19 20:40:00,02:18 +1,5hs 38min tarde,2025-01-01,1,FO,338.0,Delayed,False,20,Crítico (>3h)
4,FO 5056,Aeroparque → Mendoza,2026-02-19 16:00:00,21:27,5hs 27min tarde,2025-01-01,1,FO,327.0,Delayed,False,16,Crítico (>3h)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20181,FO 5954,Ezeiza → Florianopolis,2026-02-19 08:05:00,08:22,17min tarde,2025-12-31,12,FO,17.0,Delayed,False,8,Demora Media (15-45m)
20182,FO 5210,Ezeiza → Tucuman,2026-02-19 07:00:00,07:16,16min tarde,2025-12-31,12,FO,16.0,Delayed,False,7,Demora Media (15-45m)
20183,FO 5091,Posadas → Ezeiza,2026-02-19 14:25:00,14:40,15min tarde,2025-12-31,12,FO,15.0,Delayed,False,14,Demora Leve (0-15m)
20184,FO 5103,Iguazú → Aeroparque,2026-02-19 05:35:00,05:45,10min tarde,2025-12-31,12,FO,10.0,Delayed,False,5,Demora Leve (0-15m)


In [7]:
def unify_datasets_list(datsets_list, filename):
    df_final = pd.concat(datsets_list, ignore_index=True)
    f_name = f"{filename}.parquet"
    df_final.to_parquet(f_name)
    print("¡Todo unificado y guardado!")

In [8]:
datasets_limpios = []

datasets_limpios.append(df_jetsmart_final)
datasets_limpios.append(df_flybondi_final)

unify_datasets_list(datasets_limpios, "vuelos_historicos_consolidado")

¡Todo unificado y guardado!


Este notebook finaliza con un dataset completo unificado de las dos aerolineas que fueron scrapeadas.